# TEST MODEL 1 TANGAN

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import time

# Inisialisasi komponen utama
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1, detectionCon=0.8, minTrackCon=0.7)
classifier = Classifier("model/keras_model.h5", "model/labels.txt")

# Pengaturan untuk cropping dan resizing gambar
offset = 30
imgSize = 300
labels = ["C", "L", "V", "I"]
    
# --- Variabel Baru untuk Fungsi Penyusunan Gestur ---
sentence = ""               # Untuk menyimpan kalimat yang terbentuk
last_capture_time = 0       # Waktu terakhir gestur diambil
capture_cooldown = 2        # Jeda waktu 2 detik
capture_animation_counter = 0 # Timer untuk animasi "screenshot"

def draw_sentence_box(img, text):
    """Fungsi untuk menggambar kotak teks dan kalimat di bagian bawah gambar."""
    # Ambil dimensi gambar
    h, w, _ = img.shape
    
    # Buat overlay untuk background semi-transparan
    overlay = img.copy()
    
    # Tentukan posisi dan ukuran kotak
    box_start_point = (50, h - 110)
    box_end_point = (w - 50, h - 50)
    
    # Gambar kotak background
    cv2.rectangle(overlay, box_start_point, box_end_point, (0, 0, 0), -1)
    
    # Gabungkan overlay dengan gambar asli untuk efek transparan
    alpha = 0.5 # Tingkat transparansi
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    
    # Tambahkan teks kalimat di atas kotak
    # Menambahkan kursor sederhana '|' jika kalimat tidak kosong
    display_text = text + "_" if text else "_"
    
    cv2.putText(img, display_text, (65, h - 70), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 255, 255), 2)
    return img

# Loop utama program
while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    imgOut = img.copy()
    hands, img = detector.findHands(img, flipType=False)

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        # Pastikan area crop tidak keluar dari frame
        x_min = max(0, x - offset)
        y_min = max(0, y - offset)
        x_max = min(img.shape[1], x + w + offset)
        y_max = min(img.shape[0], y + h + offset)

        imgCrop = img[y_min:y_max, x_min:x_max]
        
        # Lanjutkan hanya jika imgCrop tidak kosong
        if imgCrop.size > 0:
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            
            # Logika resizing dengan menjaga aspect ratio
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                # Perbaikan bug: seharusnya imgSize / w
                k = imgSize / w 
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize

            # Dapatkan prediksi
            prediction, index = classifier.getPrediction(imgWhite)

            # --- Logika Baru: Ambil Gestur Setiap 2 Detik ---
            current_time = time.time()
            if current_time - last_capture_time > capture_cooldown:
                # Tambahkan label ke kalimat
                sentence += labels[index]
                # Reset timer
                last_capture_time = current_time
                # Mulai animasi screenshot
                capture_animation_counter = 5 # Animasi akan berlangsung selama 5 frame
            
            # Tampilkan label prediksi di dekat tangan
            cv2.rectangle(imgOut, (x - offset, y - offset - 40), (x - offset + 80, y - offset), (255, 0, 255), -1)
            cv2.putText(imgOut, labels[index], (x - offset + 5, y - offset - 10), cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 2)
            # cv2.imshow("White Image", imgWhite) # Opsional, bisa di-disable

    # --- Logika Baru: Animasi "Screenshot" ---
    if capture_animation_counter > 0:
        h_out, w_out, _ = imgOut.shape
        overlay = imgOut.copy()
        # Buat layar menjadi putih semi-transparan
        cv2.rectangle(overlay, (0, 0), (w_out, h_out), (255, 255, 255), -1)
        # Terapkan efeknya
        imgOut = cv2.addWeighted(overlay, 0.3, imgOut, 0.7, 0)
        # Kurangi counter animasi
        capture_animation_counter -= 1

    # --- Logika Baru: Tampilkan Kotak Kalimat ---
    imgOut = draw_sentence_box(imgOut, sentence)

    cv2.imshow("Image", imgOut)
    
    key = cv2.waitKey(1)
    # Tambahkan fitur untuk menghapus huruf terakhir dengan tombol backspace
    if key == 8 and len(sentence) > 0: # 8 adalah ASCII untuk backspace
        sentence = sentence[:-1]
    # Tambahkan fitur untuk menghapus semua kalimat dengan tombol 'c'
    elif key == ord('c'):
        sentence = ""

cap.release()
cv2.destroyAllWindows()

# Test Model 2 Tangan

In [3]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import time

# --- SETUP ---
# Initialize video capture, hand detector, and classifier
cap = cv2.VideoCapture(0)
# Set maxHands to 2 to detect both hands
detector = HandDetector(maxHands=2, detectionCon=0.8, minTrackCon=0.7) 
# IMPORTANT: Load your trained model for two-handed gestures
classifier = Classifier("model/merge/keras_model.h5", "model/merge/labels.txt")

# --- PARAMETERS ---
# Offset for the bounding box
offset = 30
# Size for the square image fed to the model
imgSize = 300
# IMPORTANT: Update this list with the labels your model can predict
labels = ["A", "B", "C","I", "L", "V"]

# --- SENTENCE BUILDING LOGIC ---
sentence = ""               # Stores the final sentence
last_capture_time = 0       # Timer to control capture rate
capture_cooldown = 2        # Cooldown in seconds between captures

def draw_sentence_box(img, text):
    """
    Draws a semi-transparent box at the bottom of the screen to display the sentence.
    """
    h, w, _ = img.shape
    overlay = img.copy()
    box_start = (50, h - 110)
    box_end = (w - 50, h - 50)
    
    cv2.rectangle(overlay, box_start, box_end, (0, 0, 0), -1)
    
    alpha = 0.6  # Transparency factor
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    
    # Add a blinking cursor effect
    display_text = text + "_"
    
    cv2.putText(img, display_text, (65, h - 70), 
                cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 255, 255), 2)
    return img

# --- MAIN LOOP ---
while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    img = cv2.flip(img, 1)
    imgOut = img.copy()
    # Find hands but don't draw on the original `img`
    hands, img = detector.findHands(img, flipType=False) 

    # --- CORE LOGIC: DETECT AND PROCESS 2 HANDS ---
    # This block runs only when exactly two hands are detected
    if len(hands) == 2:
        # Get bounding box for the first hand
        x1, y1, w1, h1 = hands[0]['bbox']
        # Get bounding box for the second hand
        x2, y2, w2, h2 = hands[1]['bbox']

        # Create a combined bounding box that covers both hands
        x_min = max(0, min(x1, x2) - offset)
        y_min = max(0, min(y1, y2) - offset)
        x_max = min(img.shape[1], max(x1 + w1, x2 + w2) + offset)
        y_max = min(img.shape[0], max(y1 + h1, y2 + h2) + offset)

        # Get the width and height of the combined box
        w_combined = x_max - x_min
        h_combined = y_max - y_min

        # Crop the image to the combined bounding box
        imgCrop = img[y_min:y_max, x_min:x_max]
        
        # Proceed only if the cropped image is valid
        if imgCrop.size > 0:
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            
            # --- RESIZE AND CENTER THE GESTURE IMAGE ---
            aspectRatio = h_combined / w_combined
            if aspectRatio > 1: # If height is greater than width
                k = imgSize / h_combined
                wCal = math.ceil(k * w_combined)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else: # If width is greater than or equal to height
                k = imgSize / w_combined
                hCal = math.ceil(k * h_combined)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize

            # --- PREDICTION AND SENTENCE LOGIC ---
            # Get prediction from the classifier
            prediction, index = classifier.getPrediction(imgWhite)

            # Check if cooldown has passed
            current_time = time.time()
            if current_time - last_capture_time > capture_cooldown:
                # Add a space if the sentence is not empty
                if sentence:
                    sentence += " "
                # Append the predicted label to the sentence
                sentence += labels[index]
                # Reset the timer
                last_capture_time = current_time
            
            # --- DISPLAY PREDICTION ON SCREEN ---
            # Draw a box and the predicted label above the combined gesture
            cv2.rectangle(imgOut, (x_min, y_min - 40), 
                          (x_min + 150, y_min), (255, 0, 255), -1)
            cv2.putText(imgOut, labels[index], (x_min + 5, y_min - 10), 
                        cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 2)
            # Draw the combined bounding box
            cv2.rectangle(imgOut, (x_min, y_min), (x_max, y_max), (255, 0, 255), 2)
            cv2.imshow("White Image", imgWhite) # Optional, can be disabled

    # --- UI AND CONTROLS ---
    # Display the sentence box on the output image
    imgOut = draw_sentence_box(imgOut, sentence)
    cv2.imshow("BISINDO Gesture Translator", imgOut)
    
    key = cv2.waitKey(1)
    # Quit with 'q'
    if key & 0xFF == ord('q'):
        break
    # Delete last word with Backspace
    if key == 8 and sentence: # 8 is ASCII for backspace
        words = sentence.split(' ')
        sentence = ' '.join(words[:-1])
    # Clear entire sentence with 'c'
    elif key == ord('c'):
        sentence = ""

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1757746845.225721  185303 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757746845.228060  187831 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-29-generic)


1/1 [==============================] - 0s 25ms/step


# FINAL TEST

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import time

# --- SETUP ---
# Inisialisasi Video Capture, Hand Detector, dan Classifier
cap = cv2.VideoCapture(0)
# PENTING: Set maxHands ke 2 agar bisa mendeteksi satu atau dua tangan
detector = HandDetector(maxHands=2, detectionCon=0.8, minTrackCon=0.7) 
# PENTING: Ganti dengan path model dan label BISINDO baru Anda
classifier = Classifier("model/merge/keras_model.h5", "model/merge/labels.txt")

# --- PARAMETER ---
offset = 30         # Offset untuk bounding box agar lebih lega
imgSize = 300       # Ukuran gambar input untuk model

# PENTING: Update list ini sesuai dengan isi file labels.txt Anda
# Pastikan urutannya sama persis dengan yang ada di file label.
labels = ["A", "B", "C","I", "L", "V"] # Contoh label, sesuaikan dengan milik Anda

# --- LOGIKA PENYUSUNAN KALIMAT ---
sentence = ""               # Menyimpan kalimat yang terbentuk
last_capture_time = 0       # Timer untuk mengontrol     kecepatan penangkapan gestur
capture_cooldown = 2        # Jeda 2 detik antar penangkapan gestur

def draw_sentence_box(img, text):
    """
    Menggambar kotak semi-transparan di bagian bawah layar untuk menampilkan kalimat.
    """
    h, w, _ = img.shape
    overlay = img.copy()
    box_start = (50, h - 110)
    box_end = (w - 50, h - 50)
    
    cv2.rectangle(overlay, box_start, box_end, (50, 50, 50), -1) # Kotak abu-abu gelap
    
    alpha = 0.6  # Faktor transparansi
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    
    # Menambahkan efek kursor berkedip sederhana
    display_text = text + "_"
    
    cv2.putText(img, display_text, (65, h - 70), 
                cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 255, 255), 2)
    return img

# --- LOOP UTAMA ---
while True:
    success, img = cap.read()
    if not success:
        print("Gagal mengambil frame dari kamera.")
        break

    img = cv2.flip(img, 1) # Flip gambar agar seperti cermin
    imgOut = img.copy()
    
    # Temukan tangan dalam gambar
    hands, img = detector.findHands(img, flipType=False) 

    imgCrop = None
    bbox_display = None
    
    # --- LOGIKA DINAMIS: CEK JUMLAH TANGAN ---
    if hands:
        # --- KASUS 1: HANYA SATU TANGAN TERDETEKSI ---
        if len(hands) == 1:
            hand = hands[0]
            x, y, w, h = hand['bbox']
            bbox_display = (x, y, w, h)
            
            # Crop gambar tangan dengan offset
            x_min = max(0, x - offset)
            y_min = max(0, y - offset)
            x_max = min(img.shape[1], x + w + offset)
            y_max = min(img.shape[0], y + h + offset)
            imgCrop = img[y_min:y_max, x_min:x_max]

        # --- KASUS 2: DUA TANGAN TERDETEKSI ---
        elif len(hands) == 2:
            # Dapatkan bounding box untuk masing-masing tangan
            x1, y1, w1, h1 = hands[0]['bbox']
            x2, y2, w2, h2 = hands[1]['bbox']

            # Buat bounding box gabungan yang mencakup kedua tangan
            x_min = max(0, min(x1, x2) - offset)
            y_min = max(0, min(y1, y2) - offset)
            x_max = min(img.shape[1], max(x1 + w1, x2 + w2) + offset)
            y_max = min(img.shape[0], max(y1 + h1, y2 + h2) + offset)
            
            bbox_display = (x_min, y_min, x_max - x_min, y_max - y_min)
            
            # Crop gambar sesuai bounding box gabungan
            imgCrop = img[y_min:y_max, x_min:x_max]
            
    # --- PROSES GAMBAR DAN PREDIKSI (JIKA TANGAN TERDETEKSI) ---
    if imgCrop is not None and imgCrop.size > 0:
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        
        # --- RESIZE DAN PUSATKAN GAMBAR GESTUR ---
        h_crop, w_crop, _ = imgCrop.shape
        aspectRatio = h_crop / w_crop
        
        if aspectRatio > 1: # Jika tinggi lebih besar dari lebar
            k = imgSize / h_crop
            wCal = math.ceil(k * w_crop)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
        else: # Jika lebar lebih besar atau sama dengan tinggi
            k = imgSize / w_crop
            hCal = math.ceil(k * h_crop)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize

        # --- PREDIKSI DAN LOGIKA KALIMAT ---
        prediction, index = classifier.getPrediction(imgWhite, draw=False)

        current_time = time.time()
        if current_time - last_capture_time > capture_cooldown:
            if sentence and labels[index] != sentence.split(" ")[-1]:
                sentence += " "
            if not sentence or labels[index] != sentence.split(" ")[-1]:
                sentence += labels[index]
            
            last_capture_time = current_time
        
        # --- TAMPILKAN PREDIKSI DI LAYAR ---
        if bbox_display:
            x, y, w, h = bbox_display
            cv2.rectangle(imgOut, (x - offset, y - offset), (x + w + offset, y + h + offset), (255, 0, 255), 2)
            cv2.rectangle(imgOut, (x - offset, y - offset - 40), (x - offset + 150, y - offset), (255, 0, 255), -1)
            cv2.putText(imgOut, labels[index], (x - offset + 5, y - offset - 10), cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 2)
            cv2.imshow("Gesture Crop", imgWhite) # Aktifkan jika ingin melihat gambar input model

    # --- UI DAN KONTROL ---
    imgOut = draw_sentence_box(imgOut, sentence)
    cv2.imshow("BISINDO Gesture Translator", imgOut)
    
    key = cv2.waitKey(1)
    # Keluar dengan tombol 'q'
    if key & 0xFF == ord('q'):
        break
    # Hapus kata terakhir dengan Backspace
    if key == 8 and sentence: # 8 adalah ASCII untuk backspace
        words = sentence.split(' ')
        sentence = ' '.join(words[:-1])
    # Hapus seluruh kalimat dengan 'c'
    elif key == ord('c'):
        sentence = ""

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1757746956.330966  185303 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1757746956.333012  191690 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-29-generic)


1/1 [==============================] - 0s 30ms/step
